
<a id='whatsnew-0140'></a>

# v0.14.0 (May 31 , 2014)

{{ header }}

This is a major release from 0.13.1 and includes a small number of API changes, several new features,
enhancements, and performance improvements along with a large number of bug fixes. We recommend that all
users upgrade to this version.

- Highlights include:  
  - Officially support Python 3.4  
  - SQL interfaces updated to use `sqlalchemy`, See [Here](#whatsnew-0140-sql).  
  - Display interface changes, See [Here](#whatsnew-0140-display)  
  - MultiIndexing Using Slicers, See [Here](#whatsnew-0140-slicers).  
  - Ability to join a singly-indexed DataFrame with a MultiIndexed DataFrame, see [Here](user_guide/merging.ipynb#merging-join-on-mi)  
  - More consistency in groupby results and more flexible groupby specifications, See [Here](#whatsnew-0140-groupby)  
  - Holiday calendars are now supported in `CustomBusinessDay`, see [Here](user_guide/timeseries.ipynb#timeseries-holiday)  
  - Several improvements in plotting functions, including: hexbin, area and pie plots, see [Here](#whatsnew-0140-plotting).  
  - Performance doc section on I/O operations, See [Here](user_guide/io.ipynb#io-perf)  
- [Other Enhancements](#whatsnew-0140-enhancements)  
- [API Changes](#whatsnew-0140-api)  
- [Text Parsing API Changes](#whatsnew-0140-parsing)  
- [Groupby API Changes](#whatsnew-0140-groupby)  
- [Performance Improvements](#whatsnew-0140-performance)  
- [Prior Deprecations](#whatsnew-0140-prior-deprecations)  
- [Deprecations](#whatsnew-0140-deprecations)  
- [Known Issues](#whatsnew-0140-knownissues)  
- [Bug Fixes](#whatsnew-0140-bug-fixes)  


In 0.14.0 all `NDFrame` based containers have undergone significant internal refactoring. Before that each block of
homogeneous data had its own labels and extra care was necessary to keep those in sync with the parent container’s labels.
This should not have any visible user/API behavior changes ([GH6745](https://github.com/pandas-dev/pandas/issues/6745))


<a id='whatsnew-0140-api'></a>

## API changes

- `read_excel` uses 0 as the default sheet ([GH6573](https://github.com/pandas-dev/pandas/issues/6573))  
- `iloc` will now accept out-of-bounds indexers for slices, e.g. a value that exceeds the length of the object being
indexed. These will be excluded. This will make pandas conform more with python/numpy indexing of out-of-bounds
values. A single indexer that is out-of-bounds and drops the dimensions of the object will still raise
`IndexError` ([GH6296](https://github.com/pandas-dev/pandas/issues/6296), [GH6299](https://github.com/pandas-dev/pandas/issues/6299)). This could result in an empty axis (e.g. an empty DataFrame being returned)  
These are out-of-bounds selections  

In [ ]:
>>> dfl.iloc[[4, 5, 6]]
IndexError: positional indexers are out-of-bounds

>>> dfl.iloc[:, 4]
IndexError: single positional indexer is out-of-bounds

- Slicing with negative start, stop & step values handles corner cases better ([GH6531](https://github.com/pandas-dev/pandas/issues/6531)):  
  - `df.iloc[:-len(df)]` is now empty  
  - `df.iloc[len(df)::-1]` now enumerates all elements in reverse  
- The `DataFrame.interpolate()` keyword `downcast` default has been changed from `infer` to
  `None`. This is to preserve the original dtype unless explicitly requested otherwise ([GH6290](https://github.com/pandas-dev/pandas/issues/6290)).  
- When converting a dataframe to HTML it used to return Empty DataFrame. This special case has
  been removed, instead a header with the column names is returned ([GH6062](https://github.com/pandas-dev/pandas/issues/6062)).  
- `Series` and `Index` now internally share more common operations, e.g. `factorize(),nunique(),value_counts()` are
  now supported on `Index` types as well. The `Series.weekday` property from is removed
  from Series for API consistency. Using a `DatetimeIndex/PeriodIndex` method on a Series will now raise a `TypeError`.
  ([GH4551](https://github.com/pandas-dev/pandas/issues/4551), [GH4056](https://github.com/pandas-dev/pandas/issues/4056), [GH5519](https://github.com/pandas-dev/pandas/issues/5519), [GH6380](https://github.com/pandas-dev/pandas/issues/6380), [GH7206](https://github.com/pandas-dev/pandas/issues/7206)).  
- Add `is_month_start`, `is_month_end`, `is_quarter_start`, `is_quarter_end`, `is_year_start`, `is_year_end` accessors for `DateTimeIndex` / `Timestamp` which return a boolean array of whether the timestamp(s) are at the start/end of the month/quarter/year defined by the frequency of the `DateTimeIndex` / `Timestamp` ([GH4565](https://github.com/pandas-dev/pandas/issues/4565), [GH6998](https://github.com/pandas-dev/pandas/issues/6998))  
- Local variable usage has changed in
  `pandas.eval()`/`DataFrame.eval()`/`DataFrame.query()`
  ([GH5987](https://github.com/pandas-dev/pandas/issues/5987)). For the `DataFrame` methods, two things have
  changed  
  - Column names are now given precedence over locals  
  - Local variables must be referred to explicitly. This means that even if
    you have a local variable that is *not* a column you must still refer to
    it with the `'@'` prefix.  
  - You can have an expression like `df.query('@a < a')` with no complaints
    from `pandas` about ambiguity of the name `a`.  
  - The top-level `pandas.eval()` function does not allow you use the
    `'@'` prefix and provides you with an error message telling you so.  
  - `NameResolutionError` was removed because it isn’t necessary anymore.  
- Define and document the order of column vs index names in query/eval ([GH6676](https://github.com/pandas-dev/pandas/issues/6676))  
- `concat` will now concatenate mixed Series and DataFrames using the Series name
  or numbering columns as needed ([GH2385](https://github.com/pandas-dev/pandas/issues/2385)). See [the docs](user_guide/merging.ipynb#merging-mixed-ndims)  
- Slicing and advanced/boolean indexing operations on `Index` classes as well
  as `Index.delete()` and `Index.drop()` methods will no longer change the type of the
  resulting index ([GH6440](https://github.com/pandas-dev/pandas/issues/6440), [GH7040](https://github.com/pandas-dev/pandas/issues/7040))  
  Previously, the above operation would return `Int64Index`.  If you’d like
  to do this manually, use `Index.astype()`  
- `set_index` no longer converts MultiIndexes to an Index of tuples. For example,
  the old behavior returned an Index in this case ([GH6459](https://github.com/pandas-dev/pandas/issues/6459)):  
  This also applies when passing multiple indices to `set_index`:  
- `pairwise` keyword was added to the statistical moment functions
`rolling_cov`, `rolling_corr`, `ewmcov`, `ewmcorr`,
`expanding_cov`, `expanding_corr` to allow the calculation of moving
window covariance and correlation matrices ([GH4950](https://github.com/pandas-dev/pandas/issues/4950)). See
[Computing rolling pairwise covariances and correlations](user_guide/computation.ipynb#stats-moments-corr-pairwise) in the docs.  

```ipython
In [1]: df = pd.DataFrame(np.random.randn(10, 4), columns=list('ABCD'))

In [4]: covs = pd.rolling_cov(df[['A', 'B', 'C']],
  ....:                       df[['B', 'C', 'D']],
  ....:                       5,
  ....:                       pairwise=True)


In [5]: covs[df.index[-1]]
Out[5]:
          B         C         D
A  0.035310  0.326593 -0.505430
B  0.137748 -0.006888 -0.005383
C -0.006888  0.861040  0.020762
```


- `Series.iteritems()` is now lazy (returns an iterator rather than a list). This was the documented behavior prior to 0.14. ([GH6760](https://github.com/pandas-dev/pandas/issues/6760))  
- Added `nunique` and `value_counts` functions to `Index` for counting unique elements. ([GH6734](https://github.com/pandas-dev/pandas/issues/6734))  
- `stack` and `unstack` now raise a `ValueError` when the `level` keyword refers
  to a non-unique item in the `Index` (previously raised a `KeyError`). ([GH6738](https://github.com/pandas-dev/pandas/issues/6738))  
- drop unused order argument from `Series.sort`; args now are in the same order as `Series.order`;
  add `na_position` arg to conform to `Series.order` ([GH6847](https://github.com/pandas-dev/pandas/issues/6847))  
- default sorting algorithm for `Series.order` is now `quicksort`, to conform with `Series.sort`
  (and numpy defaults)  
- add `inplace` keyword to `Series.order/sort` to make them inverses ([GH6859](https://github.com/pandas-dev/pandas/issues/6859))  
- `DataFrame.sort` now places NaNs at the beginning or end of the sort according to the `na_position` parameter. ([GH3917](https://github.com/pandas-dev/pandas/issues/3917))  
- accept `TextFileReader` in `concat`, which was affecting a common user idiom ([GH6583](https://github.com/pandas-dev/pandas/issues/6583)), this was a regression
  from 0.13.1  
- Added `factorize` functions to `Index` and `Series` to get indexer and unique values ([GH7090](https://github.com/pandas-dev/pandas/issues/7090))  
- `describe` on a DataFrame with a mix of Timestamp and string like objects returns a different Index ([GH7088](https://github.com/pandas-dev/pandas/issues/7088)).
  Previously the index was unintentionally sorted.  
- Arithmetic operations with **only** `bool` dtypes now give a warning indicating
that they are evaluated in Python space for `+`, `-`,
and `*` operations and raise for all others ([GH7011](https://github.com/pandas-dev/pandas/issues/7011), [GH6762](https://github.com/pandas-dev/pandas/issues/6762),
[GH7015](https://github.com/pandas-dev/pandas/issues/7015), [GH7210](https://github.com/pandas-dev/pandas/issues/7210))  

In [ ]:
>>> x = pd.Series(np.random.rand(10) > 0.5)
>>> y = True
>>> x + y  # warning generated: should do x | y instead
UserWarning: evaluating in Python space because the '+' operator is not
supported by numexpr for the bool dtype, use '|' instead
>>> x / y  # this raises because it doesn't make sense
NotImplementedError: operator '/' not implemented for bool dtypes

- In `HDFStore`, `select_as_multiple` will always raise a `KeyError`, when a key or the selector is not found ([GH6177](https://github.com/pandas-dev/pandas/issues/6177))  
- `df['col'] = value` and `df.loc[:,'col'] = value` are now completely equivalent;
  previously the `.loc` would not necessarily coerce the dtype of the resultant series ([GH6149](https://github.com/pandas-dev/pandas/issues/6149))  
- `dtypes` and `ftypes` now return a series with `dtype=object` on empty containers ([GH5740](https://github.com/pandas-dev/pandas/issues/5740))  
- `df.to_csv` will now return a string of the CSV data if neither a target path nor a buffer is provided
  ([GH6061](https://github.com/pandas-dev/pandas/issues/6061))  
- `pd.infer_freq()` will now raise a `TypeError` if given an invalid `Series/Index`
  type ([GH6407](https://github.com/pandas-dev/pandas/issues/6407), [GH6463](https://github.com/pandas-dev/pandas/issues/6463))  
- A tuple passed to `DataFame.sort_index` will be interpreted as the levels of
  the index, rather than requiring a list of tuple ([GH4370](https://github.com/pandas-dev/pandas/issues/4370))  
- all offset operations now return `Timestamp` types (rather than datetime), Business/Week frequencies were incorrect ([GH4069](https://github.com/pandas-dev/pandas/issues/4069))  
- `to_excel` now converts `np.inf` into a string representation,
  customizable by the `inf_rep` keyword argument (Excel has no native inf
  representation) ([GH6782](https://github.com/pandas-dev/pandas/issues/6782))  
- Replace `pandas.compat.scipy.scoreatpercentile` with `numpy.percentile` ([GH6810](https://github.com/pandas-dev/pandas/issues/6810))  
- `.quantile` on a `datetime[ns]` series now returns `Timestamp` instead
  of `np.datetime64` objects ([GH6810](https://github.com/pandas-dev/pandas/issues/6810))  
- change `AssertionError` to `TypeError` for invalid types passed to `concat` ([GH6583](https://github.com/pandas-dev/pandas/issues/6583))  
- Raise a `TypeError` when `DataFrame` is passed an iterator as the
  `data` argument ([GH5357](https://github.com/pandas-dev/pandas/issues/5357))  



<a id='whatsnew-0140-display'></a>

## Display changes

- The default way of printing large DataFrames has changed. DataFrames
  exceeding `max_rows` and/or `max_columns` are now displayed in a
  centrally truncated view, consistent with the printing of a
  `pandas.Series` ([GH5603](https://github.com/pandas-dev/pandas/issues/5603)).  
  In previous versions, a DataFrame was truncated once the dimension
  constraints were reached and an ellipse (…) signaled that part of
  the data was cut off.  
  <img src="whatsnew/../_static/trunc_before.png" alt="The previous look of truncate." style="">
  
  In the current version, large DataFrames are centrally truncated,
  showing a preview of head and tail in both dimensions.  
  <img src="whatsnew/../_static/trunc_after.png" alt="The new look." style="">
  
- allow option `'truncate'` for `display.show_dimensions` to only show the dimensions if the
  frame is truncated ([GH6547](https://github.com/pandas-dev/pandas/issues/6547)).  
  The default for `display.show_dimensions` will now be `truncate`. This is consistent with
  how Series display length.  
- Regression in the display of a MultiIndexed Series with `display.max_rows` is less than the
  length of the series ([GH7101](https://github.com/pandas-dev/pandas/issues/7101))  
- Fixed a bug in the HTML repr of a truncated Series or DataFrame not showing the class name with the
  large_repr set to ‘info’ ([GH7105](https://github.com/pandas-dev/pandas/issues/7105))  
- The verbose keyword in `DataFrame.info()`, which controls whether to shorten the `info`
  representation, is now `None` by default. This will follow the global setting in
  `display.max_info_columns`. The global setting can be overridden with `verbose=True` or
  `verbose=False`.  
- Fixed a bug with the info repr not honoring the display.max_info_columns setting ([GH6939](https://github.com/pandas-dev/pandas/issues/6939))  
- Offset/freq info now in Timestamp __repr__ ([GH4553](https://github.com/pandas-dev/pandas/issues/4553))  



<a id='whatsnew-0140-parsing'></a>

## Text parsing API changes

`read_csv()`/`read_table()` will now be noisier w.r.t invalid options rather than falling back to the `PythonParser`.

- Raise `ValueError` when `sep` specified with
  `delim_whitespace=True` in `read_csv()`/`read_table()`
  ([GH6607](https://github.com/pandas-dev/pandas/issues/6607))  
- Raise `ValueError` when `engine='c'` specified with unsupported
  options in `read_csv()`/`read_table()` ([GH6607](https://github.com/pandas-dev/pandas/issues/6607))  
- Raise `ValueError` when fallback to python parser causes options to be
  ignored ([GH6607](https://github.com/pandas-dev/pandas/issues/6607))  
- Produce `ParserWarning` on fallback to python
  parser when no options are ignored ([GH6607](https://github.com/pandas-dev/pandas/issues/6607))  
- Translate `sep='\s+'` to `delim_whitespace=True` in
  `read_csv()`/`read_table()` if no other C-unsupported options
  specified ([GH6607](https://github.com/pandas-dev/pandas/issues/6607))  



<a id='whatsnew-0140-groupby'></a>

## Groupby API changes

More consistent behavior for some groupby methods:

- groupby `head` and `tail` now act more like `filter` rather than an aggregation:  
- groupby head and tail respect column selection:  
- groupby `nth` now reduces by default; filtering can be achieved by passing `as_index=False`. With an optional `dropna` argument to ignore
  NaN. See [the docs](user_guide/groupby.ipynb#groupby-nth).  
  Reducing  
  Filtering  
- groupby will now not return the grouped column for non-cython functions ([GH5610](https://github.com/pandas-dev/pandas/issues/5610), [GH5614](https://github.com/pandas-dev/pandas/issues/5614), [GH6732](https://github.com/pandas-dev/pandas/issues/6732)),
  as its already the index  
- passing `as_index` will leave the grouped column in-place (this is not change in 0.14.0)  
- Allow specification of a more complex groupby via `pd.Grouper`, such as grouping
  by a Time and a string field simultaneously. See [the docs](user_guide/groupby.ipynb#groupby-specify). ([GH3794](https://github.com/pandas-dev/pandas/issues/3794))  
- Better propagation/preservation of Series names when performing groupby
  operations:  
  - `SeriesGroupBy.agg` will ensure that the name attribute of the original
    series is propagated to the result ([GH6265](https://github.com/pandas-dev/pandas/issues/6265)).  
  - If the function provided to `GroupBy.apply` returns a named series, the
    name of the series will be kept as the name of the column index of the
    DataFrame returned by `GroupBy.apply` ([GH6124](https://github.com/pandas-dev/pandas/issues/6124)).  This facilitates
    `DataFrame.stack` operations where the name of the column index is used as
    the name of the inserted column containing the pivoted data.  



<a id='whatsnew-0140-sql'></a>

## SQL

The SQL reading and writing functions now support more database flavors
through SQLAlchemy ([GH2717](https://github.com/pandas-dev/pandas/issues/2717), [GH4163](https://github.com/pandas-dev/pandas/issues/4163), [GH5950](https://github.com/pandas-dev/pandas/issues/5950), [GH6292](https://github.com/pandas-dev/pandas/issues/6292)).
All databases supported by SQLAlchemy can be used, such
as PostgreSQL, MySQL, Oracle, Microsoft SQL server (see documentation of
SQLAlchemy on [included dialects](https://sqlalchemy.readthedocs.io/en/latest/dialects/index.html)).

The functionality of providing DBAPI connection objects will only be supported
for sqlite3 in the future. The `'mysql'` flavor is deprecated.

The new functions [`read_sql_query()`](reference/api/pandas.read_sql_query.ipynb#pandas.read_sql_query) and [`read_sql_table()`](reference/api/pandas.read_sql_table.ipynb#pandas.read_sql_table)
are introduced. The function [`read_sql()`](reference/api/pandas.read_sql.ipynb#pandas.read_sql) is kept as a convenience
wrapper around the other two and will delegate to specific function depending on
the provided input (database table name or sql query).

In practice, you have to provide a SQLAlchemy `engine` to the sql functions.
To connect with SQLAlchemy you use the `create_engine()` function to create an engine
object from database URI. You only need to create the engine once per database you are
connecting to. For an in-memory sqlite database:

This `engine` can then be used to write or read data to/from this database:

You can read data from a database by specifying the table name:

or by specifying a sql query:

Some other enhancements to the sql functions include:

- support for writing the index. This can be controlled with the `index`
  keyword (default is True).  
- specify the column label to use when writing the index with `index_label`.  
- specify string columns to parse as datetimes with the `parse_dates`
  keyword in [`read_sql_query()`](reference/api/pandas.read_sql_query.ipynb#pandas.read_sql_query) and [`read_sql_table()`](reference/api/pandas.read_sql_table.ipynb#pandas.read_sql_table).  


Some of the existing functions or function aliases have been deprecated
and will be removed in future versions. This includes: `tquery`, `uquery`,
`read_frame`, `frame_query`, `write_frame`.

The support for the ‘mysql’ flavor when using DBAPI connection objects has been deprecated.
MySQL will be further supported with SQLAlchemy engines ([GH6900](https://github.com/pandas-dev/pandas/issues/6900)).


<a id='whatsnew-0140-slicers'></a>

## MultiIndexing using slicers

In 0.14.0 we added a new way to slice MultiIndexed objects.
You can slice a MultiIndex by providing multiple indexers.

You can provide any of the selectors as if you are indexing by label, see [Selection by Label](user_guide/indexing.ipynb#indexing-label),
including slices, lists of labels, labels, and boolean indexers.

You can use `slice(None)` to select all the contents of *that* level. You do not need to specify all the
*deeper* levels, they will be implied as `slice(None)`.

As usual, **both sides** of the slicers are included as this is label indexing.

See [the docs](user_guide/advanced.ipynb#advanced-mi-slicers)
See also issues ([GH6134](https://github.com/pandas-dev/pandas/issues/6134), [GH4036](https://github.com/pandas-dev/pandas/issues/4036), [GH3057](https://github.com/pandas-dev/pandas/issues/3057), [GH2598](https://github.com/pandas-dev/pandas/issues/2598), [GH5641](https://github.com/pandas-dev/pandas/issues/5641), [GH7106](https://github.com/pandas-dev/pandas/issues/7106))

> You should specify all axes in the `.loc` specifier, meaning the indexer for the **index** and
for the **columns**. Their are some ambiguous cases where the passed indexer could be mis-interpreted
as indexing *both* axes, rather than into say the MuliIndex for the rows.

You should do this:

In [ ]:
  >>> df.loc[(slice('A1', 'A3'), ...), :]  # noqa: E901

rather than this:

In [ ]:
>>> df.loc[(slice('A1', 'A3'), ...)]  # noqa: E901

You will need to make sure that the selection axes are fully lexsorted!

Basic MultiIndex slicing using slices, lists, and labels.

You can use a `pd.IndexSlice` to shortcut the creation of these slices

It is possible to perform quite complicated selections using this method on multiple
axes at the same time.

Using a boolean indexer you can provide selection related to the *values*.

You can also specify the `axis` argument to `.loc` to interpret the passed
slicers on a single axis.

Furthermore you can *set* the values using these methods

You can use a right-hand-side of an alignable object as well.


<a id='whatsnew-0140-plotting'></a>

## Plotting

- Hexagonal bin plots from `DataFrame.plot` with `kind='hexbin'` ([GH5478](https://github.com/pandas-dev/pandas/issues/5478)), See [the docs](user_guide/visualization.ipynb#visualization-hexbin).  
- `DataFrame.plot` and `Series.plot` now supports area plot with specifying `kind='area'` ([GH6656](https://github.com/pandas-dev/pandas/issues/6656)), See [the docs](user_guide/visualization.ipynb#visualization-area-plot)  
- Pie plots from `Series.plot` and `DataFrame.plot` with `kind='pie'` ([GH6976](https://github.com/pandas-dev/pandas/issues/6976)), See [the docs](user_guide/visualization.ipynb#visualization-pie).  
- Plotting with Error Bars is now supported in the `.plot` method of `DataFrame` and `Series` objects ([GH3796](https://github.com/pandas-dev/pandas/issues/3796), [GH6834](https://github.com/pandas-dev/pandas/issues/6834)), See [the docs](user_guide/visualization.ipynb#visualization-errorbars).  
- `DataFrame.plot` and `Series.plot` now support a `table` keyword for plotting `matplotlib.Table`, See [the docs](user_guide/visualization.ipynb#visualization-table). The `table` keyword can receive the following values.  
  - `False`: Do nothing (default).  
  - `True`: Draw a table using the `DataFrame` or `Series` called `plot` method. Data will be transposed to meet matplotlib’s default layout.  
  - `DataFrame` or `Series`: Draw matplotlib.table using the passed data. The data will be drawn as displayed in print method (not transposed automatically).
    Also, helper function `pandas.tools.plotting.table` is added to create a table from `DataFrame` and `Series`, and add it to an `matplotlib.Axes`.  
- `plot(legend='reverse')` will now reverse the order of legend labels for
  most plot kinds. ([GH6014](https://github.com/pandas-dev/pandas/issues/6014))  
- Line plot and area plot can be stacked by `stacked=True` ([GH6656](https://github.com/pandas-dev/pandas/issues/6656))  
- Following keywords are now acceptable for `DataFrame.plot()` with `kind='bar'` and `kind='barh'`:  
  - width: Specify the bar width. In previous versions, static value 0.5 was passed to matplotlib and it cannot be overwritten. ([GH6604](https://github.com/pandas-dev/pandas/issues/6604))  
  - align: Specify the bar alignment. Default is center (different from matplotlib). In previous versions, pandas passes align=’edge’ to matplotlib and adjust the location to center by itself, and it results align keyword is not applied as expected. ([GH4525](https://github.com/pandas-dev/pandas/issues/4525))  
  - position: Specify relative alignments for bar plot layout. From 0 (left/bottom-end) to 1(right/top-end). Default is 0.5 (center). ([GH6604](https://github.com/pandas-dev/pandas/issues/6604))  
  Because of the default align value changes, coordinates of bar plots are now located on integer values (0.0, 1.0, 2.0 …). This is intended to make bar plot be located on the same coordinates as line plot. However, bar plot may differs unexpectedly when you manually adjust the bar location or drawing area, such as using set_xlim, set_ylim, etc. In this cases, please modify your script to meet with new coordinates.  
- The `parallel_coordinates()` function now takes argument `color`
  instead of `colors`. A `FutureWarning` is raised to alert that
  the old `colors` argument will not be supported in a future release. ([GH6956](https://github.com/pandas-dev/pandas/issues/6956))  
- The `parallel_coordinates()` and `andrews_curves()` functions now take
  positional argument `frame` instead of `data`. A `FutureWarning` is
  raised if the old `data` argument is used by name. ([GH6956](https://github.com/pandas-dev/pandas/issues/6956))  
- `DataFrame.boxplot()` now supports `layout` keyword ([GH6769](https://github.com/pandas-dev/pandas/issues/6769))  
- `DataFrame.boxplot()` has a new keyword argument, return_type. It accepts `'dict'`,
  `'axes'`, or `'both'`, in which case a namedtuple with the matplotlib
  axes and a dict of matplotlib Lines is returned.  



<a id='whatsnew-0140-prior-deprecations'></a>

## Prior version deprecations/changes

There are prior version deprecations that are taking effect as of 0.14.0.

- Remove `DateRange` in favor of `DatetimeIndex` ([GH6816](https://github.com/pandas-dev/pandas/issues/6816))  
- Remove `column` keyword from `DataFrame.sort` ([GH4370](https://github.com/pandas-dev/pandas/issues/4370))  
- Remove `precision` keyword from `set_eng_float_format()` ([GH395](https://github.com/pandas-dev/pandas/issues/395))  
- Remove `force_unicode` keyword from `DataFrame.to_string()`,
  `DataFrame.to_latex()`, and `DataFrame.to_html()`; these function
  encode in unicode by default ([GH2224](https://github.com/pandas-dev/pandas/issues/2224), [GH2225](https://github.com/pandas-dev/pandas/issues/2225))  
- Remove `nanRep` keyword from `DataFrame.to_csv()` and
  `DataFrame.to_string()` ([GH275](https://github.com/pandas-dev/pandas/issues/275))  
- Remove `unique` keyword from `HDFStore.select_column()` ([GH3256](https://github.com/pandas-dev/pandas/issues/3256))  
- Remove `inferTimeRule` keyword from `Timestamp.offset()` ([GH391](https://github.com/pandas-dev/pandas/issues/391))  
- Remove `name` keyword from `get_data_yahoo()` and
  `get_data_google()` ( [commit b921d1a](https://github.com/pandas-dev/pandas/commit/b921d1a2) )  
- Remove `offset` keyword from `DatetimeIndex` constructor
  ( [commit 3136390](https://github.com/pandas-dev/pandas/commit/3136390) )  
- Remove `time_rule` from several rolling-moment statistical functions, such
  as `rolling_sum()` ([GH1042](https://github.com/pandas-dev/pandas/issues/1042))  
- Removed neg `-` boolean operations on numpy arrays in favor of inv `~`, as this is going to
  be deprecated in numpy 1.9 ([GH6960](https://github.com/pandas-dev/pandas/issues/6960))  



<a id='whatsnew-0140-deprecations'></a>

## Deprecations

- The `pivot_table()`/`DataFrame.pivot_table()` and `crosstab()` functions
  now take arguments `index` and `columns` instead of `rows` and `cols`.  A
  `FutureWarning` is raised to alert that the old `rows` and `cols` arguments
  will not be supported in a future release ([GH5505](https://github.com/pandas-dev/pandas/issues/5505))  
- The `DataFrame.drop_duplicates()` and `DataFrame.duplicated()` methods
  now take argument `subset` instead of `cols` to better align with
  `DataFrame.dropna()`.  A `FutureWarning` is raised to alert that the old
  `cols` arguments will not be supported in a future release ([GH6680](https://github.com/pandas-dev/pandas/issues/6680))  
- The `DataFrame.to_csv()` and `DataFrame.to_excel()` functions
  now takes argument `columns` instead of `cols`.  A
  `FutureWarning` is raised to alert that the old `cols` arguments
  will not be supported in a future release ([GH6645](https://github.com/pandas-dev/pandas/issues/6645))  
- Indexers will warn `FutureWarning` when used with a scalar indexer and
a non-floating point Index ([GH4892](https://github.com/pandas-dev/pandas/issues/4892), [GH6960](https://github.com/pandas-dev/pandas/issues/6960))  

```ipython
# non-floating point indexes can only be indexed by integers / labels
In [1]: pd.Series(1, np.arange(5))[3.0]
        pandas/core/index.py:469: FutureWarning: scalar indexers for index type Int64Index should be integers and not floating point
Out[1]: 1

In [2]: pd.Series(1, np.arange(5)).iloc[3.0]
        pandas/core/index.py:469: FutureWarning: scalar indexers for index type Int64Index should be integers and not floating point
Out[2]: 1

In [3]: pd.Series(1, np.arange(5)).iloc[3.0:4]
        pandas/core/index.py:527: FutureWarning: slice indexers when using iloc should be integers and not floating point
Out[3]:
        3    1
        dtype: int64

# these are Float64Indexes, so integer or floating point is acceptable
In [4]: pd.Series(1, np.arange(5.))[3]
Out[4]: 1

In [5]: pd.Series(1, np.arange(5.))[3.0]
Out[6]: 1
```


- Numpy 1.9 compat w.r.t. deprecation warnings ([GH6960](https://github.com/pandas-dev/pandas/issues/6960))  
- `Panel.shift()` now has a function signature that matches `DataFrame.shift()`.
  The old positional argument `lags` has been changed to a keyword argument
  `periods` with a default value of 1. A `FutureWarning` is raised if the
  old argument `lags` is used by name. ([GH6910](https://github.com/pandas-dev/pandas/issues/6910))  
- The `order` keyword argument of `factorize()` will be removed. ([GH6926](https://github.com/pandas-dev/pandas/issues/6926)).  
- Remove the `copy` keyword from `DataFrame.xs()`, `Panel.major_xs()`, `Panel.minor_xs()`. A view will be
  returned if possible, otherwise a copy will be made. Previously the user could think that `copy=False` would
  ALWAYS return a view. ([GH6894](https://github.com/pandas-dev/pandas/issues/6894))  
- The `parallel_coordinates()` function now takes argument `color`
  instead of `colors`. A `FutureWarning` is raised to alert that
  the old `colors` argument will not be supported in a future release. ([GH6956](https://github.com/pandas-dev/pandas/issues/6956))  
- The `parallel_coordinates()` and `andrews_curves()` functions now take
  positional argument `frame` instead of `data`. A `FutureWarning` is
  raised if the old `data` argument is used by name. ([GH6956](https://github.com/pandas-dev/pandas/issues/6956))  
- The support for the ‘mysql’ flavor when using DBAPI connection objects has been deprecated.
  MySQL will be further supported with SQLAlchemy engines ([GH6900](https://github.com/pandas-dev/pandas/issues/6900)).  
- The following `io.sql` functions have been deprecated: `tquery`, `uquery`, `read_frame`, `frame_query`, `write_frame`.  
- The percentile_width keyword argument in `describe()` has been deprecated.
  Use the percentiles keyword instead, which takes a list of percentiles to display. The
  default output is unchanged.  
- The default return type of `boxplot()` will change from a dict to a matplotlib Axes
  in a future release. You can use the future behavior now by passing `return_type='axes'`
  to boxplot.  



<a id='whatsnew-0140-knownissues'></a>

## Known issues

- OpenPyXL 2.0.0 breaks backwards compatibility ([GH7169](https://github.com/pandas-dev/pandas/issues/7169))  



<a id='whatsnew-0140-enhancements'></a>

## Enhancements

- DataFrame and Series will create a MultiIndex object if passed a tuples dict, See [the docs](getting_started/dsintro.ipynb#basics-dataframe-from-dict-of-tuples) ([GH3323](https://github.com/pandas-dev/pandas/issues/3323))  
- Added the `sym_diff` method to `Index` ([GH5543](https://github.com/pandas-dev/pandas/issues/5543))  
- `DataFrame.to_latex` now takes a longtable keyword, which if True will return a table in a longtable environment. ([GH6617](https://github.com/pandas-dev/pandas/issues/6617))  
- Add option to turn off escaping in `DataFrame.to_latex` ([GH6472](https://github.com/pandas-dev/pandas/issues/6472))  
- `pd.read_clipboard` will, if the keyword `sep` is unspecified, try to detect data copied from a spreadsheet
  and parse accordingly. ([GH6223](https://github.com/pandas-dev/pandas/issues/6223))  
- Joining a singly-indexed DataFrame with a MultiIndexed DataFrame ([GH3662](https://github.com/pandas-dev/pandas/issues/3662))  
  See [the docs](user_guide/merging.ipynb#merging-join-on-mi). Joining MultiIndex DataFrames on both the left and right is not yet supported ATM.  
- `quotechar`, `doublequote`, and `escapechar` can now be specified when
  using `DataFrame.to_csv` ([GH5414](https://github.com/pandas-dev/pandas/issues/5414), [GH4528](https://github.com/pandas-dev/pandas/issues/4528))  
- Partially sort by only the specified levels of a MultiIndex with the
  `sort_remaining` boolean kwarg. ([GH3984](https://github.com/pandas-dev/pandas/issues/3984))  
- Added `to_julian_date` to `TimeStamp` and `DatetimeIndex`.  The Julian
  Date is used primarily in astronomy and represents the number of days from
  noon, January 1, 4713 BC.  Because nanoseconds are used to define the time
  in pandas the actual range of dates that you can use is 1678 AD to 2262 AD. ([GH4041](https://github.com/pandas-dev/pandas/issues/4041))  
- `DataFrame.to_stata` will now check data for compatibility with Stata data types
  and will upcast when needed.  When it is not possible to losslessly upcast, a warning
  is issued ([GH6327](https://github.com/pandas-dev/pandas/issues/6327))  
- `DataFrame.to_stata` and `StataWriter` will accept keyword arguments time_stamp
  and data_label which allow the time stamp and dataset label to be set when creating a
  file. ([GH6545](https://github.com/pandas-dev/pandas/issues/6545))  
- `pandas.io.gbq` now handles reading unicode strings properly. ([GH5940](https://github.com/pandas-dev/pandas/issues/5940))  
- [Holidays Calendars](user_guide/timeseries.ipynb#timeseries-holiday) are now available and can be used with the `CustomBusinessDay` offset ([GH6719](https://github.com/pandas-dev/pandas/issues/6719))  
- `Float64Index` is now backed by a `float64` dtype ndarray instead of an
  `object` dtype array ([GH6471](https://github.com/pandas-dev/pandas/issues/6471)).  
- Implemented `Panel.pct_change` ([GH6904](https://github.com/pandas-dev/pandas/issues/6904))  
- Added `how` option to rolling-moment functions to dictate how to handle resampling; `rolling_max()` defaults to max,
  `rolling_min()` defaults to min, and all others default to mean ([GH6297](https://github.com/pandas-dev/pandas/issues/6297))  
- `CustomBusinessMonthBegin` and `CustomBusinessMonthEnd` are now available ([GH6866](https://github.com/pandas-dev/pandas/issues/6866))  
- `Series.quantile()` and `DataFrame.quantile()` now accept an array of
  quantiles.  
- `describe()` now accepts an array of percentiles to include in the summary statistics ([GH4196](https://github.com/pandas-dev/pandas/issues/4196))  
- `pivot_table` can now accept `Grouper` by `index` and `columns` keywords ([GH6913](https://github.com/pandas-dev/pandas/issues/6913))  
- Arrays of strings can be wrapped to a specified width (`str.wrap`) ([GH6999](https://github.com/pandas-dev/pandas/issues/6999))  
- Add `nsmallest()` and `Series.nlargest()` methods to Series, See [the docs](getting_started/basics.ipynb#basics-nsorted) ([GH3960](https://github.com/pandas-dev/pandas/issues/3960))  
- PeriodIndex fully supports partial string indexing like DatetimeIndex ([GH7043](https://github.com/pandas-dev/pandas/issues/7043))  
- `read_excel` can now read milliseconds in Excel dates and times with xlrd >= 0.9.3. ([GH5945](https://github.com/pandas-dev/pandas/issues/5945))  
- `pd.stats.moments.rolling_var` now uses Welford’s method for increased numerical stability ([GH6817](https://github.com/pandas-dev/pandas/issues/6817))  
- pd.expanding_apply and pd.rolling_apply now take args and kwargs that are passed on to
  the func ([GH6289](https://github.com/pandas-dev/pandas/issues/6289))  
- `DataFrame.rank()` now has a percentage rank option ([GH5971](https://github.com/pandas-dev/pandas/issues/5971))  
- `Series.rank()` now has a percentage rank option ([GH5971](https://github.com/pandas-dev/pandas/issues/5971))  
- `Series.rank()` and `DataFrame.rank()` now accept `method='dense'` for ranks without gaps ([GH6514](https://github.com/pandas-dev/pandas/issues/6514))  
- Support passing `encoding` with xlwt ([GH3710](https://github.com/pandas-dev/pandas/issues/3710))  
- Refactor Block classes removing Block.items attributes to avoid duplication
  in item handling ([GH6745](https://github.com/pandas-dev/pandas/issues/6745), [GH6988](https://github.com/pandas-dev/pandas/issues/6988)).  
- Testing statements updated to use specialized asserts ([GH6175](https://github.com/pandas-dev/pandas/issues/6175))  



<a id='whatsnew-0140-performance'></a>

## Performance

- Performance improvement when converting `DatetimeIndex` to floating ordinals
  using `DatetimeConverter` ([GH6636](https://github.com/pandas-dev/pandas/issues/6636))  
- Performance improvement for  `DataFrame.shift` ([GH5609](https://github.com/pandas-dev/pandas/issues/5609))  
- Performance improvement in indexing into a MultiIndexed Series ([GH5567](https://github.com/pandas-dev/pandas/issues/5567))  
- Performance improvements in single-dtyped indexing ([GH6484](https://github.com/pandas-dev/pandas/issues/6484))  
- Improve performance of DataFrame construction with certain offsets, by removing faulty caching
  (e.g. MonthEnd,BusinessMonthEnd), ([GH6479](https://github.com/pandas-dev/pandas/issues/6479))  
- Improve performance of `CustomBusinessDay` ([GH6584](https://github.com/pandas-dev/pandas/issues/6584))  
- improve performance of slice indexing on Series with string keys ([GH6341](https://github.com/pandas-dev/pandas/issues/6341), [GH6372](https://github.com/pandas-dev/pandas/issues/6372))  
- Performance improvement for `DataFrame.from_records` when reading a
  specified number of rows from an iterable ([GH6700](https://github.com/pandas-dev/pandas/issues/6700))  
- Performance improvements in timedelta conversions for integer dtypes ([GH6754](https://github.com/pandas-dev/pandas/issues/6754))  
- Improved performance of compatible pickles ([GH6899](https://github.com/pandas-dev/pandas/issues/6899))  
- Improve performance in certain reindexing operations by optimizing `take_2d` ([GH6749](https://github.com/pandas-dev/pandas/issues/6749))  
- `GroupBy.count()` is now implemented in Cython and is much faster for large
  numbers of groups ([GH7016](https://github.com/pandas-dev/pandas/issues/7016)).  

## Experimental

There are no experimental changes in 0.14.0


<a id='whatsnew-0140-bug-fixes'></a>

## Bug Fixes

- Bug in Series ValueError when index doesn’t match data ([GH6532](https://github.com/pandas-dev/pandas/issues/6532))  
- Prevent segfault due to MultiIndex not being supported in HDFStore table
  format ([GH1848](https://github.com/pandas-dev/pandas/issues/1848))  
- Bug in `pd.DataFrame.sort_index` where mergesort wasn’t stable when `ascending=False` ([GH6399](https://github.com/pandas-dev/pandas/issues/6399))  
- Bug in `pd.tseries.frequencies.to_offset` when argument has leading zeros ([GH6391](https://github.com/pandas-dev/pandas/issues/6391))  
- Bug in version string gen. for dev versions with shallow clones / install from tarball ([GH6127](https://github.com/pandas-dev/pandas/issues/6127))  
- Inconsistent tz parsing `Timestamp` / `to_datetime` for current year ([GH5958](https://github.com/pandas-dev/pandas/issues/5958))  
- Indexing bugs with reordered indexes ([GH6252](https://github.com/pandas-dev/pandas/issues/6252), [GH6254](https://github.com/pandas-dev/pandas/issues/6254))  
- Bug in `.xs` with a Series multiindex ([GH6258](https://github.com/pandas-dev/pandas/issues/6258), [GH5684](https://github.com/pandas-dev/pandas/issues/5684))  
- Bug in conversion of a string types to a DatetimeIndex with a specified frequency ([GH6273](https://github.com/pandas-dev/pandas/issues/6273), [GH6274](https://github.com/pandas-dev/pandas/issues/6274))  
- Bug in `eval` where type-promotion failed for large expressions ([GH6205](https://github.com/pandas-dev/pandas/issues/6205))  
- Bug in interpolate with `inplace=True` ([GH6281](https://github.com/pandas-dev/pandas/issues/6281))  
- `HDFStore.remove` now handles start and stop ([GH6177](https://github.com/pandas-dev/pandas/issues/6177))  
- `HDFStore.select_as_multiple` handles start and stop the same way as `select` ([GH6177](https://github.com/pandas-dev/pandas/issues/6177))  
- `HDFStore.select_as_coordinates` and `select_column` works with a `where` clause that results in filters ([GH6177](https://github.com/pandas-dev/pandas/issues/6177))  
- Regression in join of non_unique_indexes ([GH6329](https://github.com/pandas-dev/pandas/issues/6329))  
- Issue with groupby `agg` with a single function and a a mixed-type frame ([GH6337](https://github.com/pandas-dev/pandas/issues/6337))  
- Bug in `DataFrame.replace()` when passing a non- `bool`
  `to_replace` argument ([GH6332](https://github.com/pandas-dev/pandas/issues/6332))  
- Raise when trying to align on different levels of a MultiIndex assignment ([GH3738](https://github.com/pandas-dev/pandas/issues/3738))  
- Bug in setting complex dtypes via boolean indexing ([GH6345](https://github.com/pandas-dev/pandas/issues/6345))  
- Bug in TimeGrouper/resample when presented with a non-monotonic DatetimeIndex that would return invalid results. ([GH4161](https://github.com/pandas-dev/pandas/issues/4161))  
- Bug in index name propagation in TimeGrouper/resample ([GH4161](https://github.com/pandas-dev/pandas/issues/4161))  
- TimeGrouper has a more compatible API to the rest of the groupers (e.g. `groups` was missing) ([GH3881](https://github.com/pandas-dev/pandas/issues/3881))  
- Bug in multiple grouping with a TimeGrouper depending on target column order ([GH6764](https://github.com/pandas-dev/pandas/issues/6764))  
- Bug in `pd.eval` when parsing strings with possible tokens like `'&'`
  ([GH6351](https://github.com/pandas-dev/pandas/issues/6351))  
- Bug correctly handle placements of `-inf` in Panels when dividing by integer 0 ([GH6178](https://github.com/pandas-dev/pandas/issues/6178))  
- `DataFrame.shift` with `axis=1` was raising ([GH6371](https://github.com/pandas-dev/pandas/issues/6371))  
- Disabled clipboard tests until release time (run locally with `nosetests -A disabled`) ([GH6048](https://github.com/pandas-dev/pandas/issues/6048)).  
- Bug in `DataFrame.replace()` when passing a nested `dict` that contained
  keys not in the values to be replaced ([GH6342](https://github.com/pandas-dev/pandas/issues/6342))  
- `str.match` ignored the na flag ([GH6609](https://github.com/pandas-dev/pandas/issues/6609)).  
- Bug in take with duplicate columns that were not consolidated ([GH6240](https://github.com/pandas-dev/pandas/issues/6240))  
- Bug in interpolate changing dtypes ([GH6290](https://github.com/pandas-dev/pandas/issues/6290))  
- Bug in `Series.get`, was using a buggy access method ([GH6383](https://github.com/pandas-dev/pandas/issues/6383))  
- Bug in hdfstore queries of the form `where=[('date', '>=', datetime(2013,1,1)), ('date', '<=', datetime(2014,1,1))]` ([GH6313](https://github.com/pandas-dev/pandas/issues/6313))  
- Bug in `DataFrame.dropna` with duplicate indices ([GH6355](https://github.com/pandas-dev/pandas/issues/6355))  
- Regression in chained getitem indexing with embedded list-like from 0.12 ([GH6394](https://github.com/pandas-dev/pandas/issues/6394))  
- `Float64Index` with nans not comparing correctly ([GH6401](https://github.com/pandas-dev/pandas/issues/6401))  
- `eval`/`query` expressions with strings containing the `@` character
  will now work ([GH6366](https://github.com/pandas-dev/pandas/issues/6366)).  
- Bug in `Series.reindex` when specifying a `method` with some nan values was inconsistent (noted on a resample) ([GH6418](https://github.com/pandas-dev/pandas/issues/6418))  
- Bug in `DataFrame.replace()` where nested dicts were erroneously
  depending on the order of dictionary keys and values ([GH5338](https://github.com/pandas-dev/pandas/issues/5338)).  
- Performance issue in concatenating with empty objects ([GH3259](https://github.com/pandas-dev/pandas/issues/3259))  
- Clarify sorting of `sym_diff` on `Index` objects with `NaN` values ([GH6444](https://github.com/pandas-dev/pandas/issues/6444))  
- Regression in `MultiIndex.from_product` with a `DatetimeIndex` as input ([GH6439](https://github.com/pandas-dev/pandas/issues/6439))  
- Bug in `str.extract` when passed a non-default index ([GH6348](https://github.com/pandas-dev/pandas/issues/6348))  
- Bug in `str.split` when passed `pat=None` and `n=1` ([GH6466](https://github.com/pandas-dev/pandas/issues/6466))  
- Bug in `io.data.DataReader` when passed `"F-F_Momentum_Factor"` and `data_source="famafrench"` ([GH6460](https://github.com/pandas-dev/pandas/issues/6460))  
- Bug in `sum` of a `timedelta64[ns]` series ([GH6462](https://github.com/pandas-dev/pandas/issues/6462))  
- Bug in `resample` with a timezone and certain offsets ([GH6397](https://github.com/pandas-dev/pandas/issues/6397))  
- Bug in `iat/iloc` with duplicate indices on a Series ([GH6493](https://github.com/pandas-dev/pandas/issues/6493))  
- Bug in `read_html` where nan’s were incorrectly being used to indicate
  missing values in text. Should use the empty string for consistency with the
  rest of pandas ([GH5129](https://github.com/pandas-dev/pandas/issues/5129)).  
- Bug in `read_html` tests where redirected invalid URLs would make one test
  fail ([GH6445](https://github.com/pandas-dev/pandas/issues/6445)).  
- Bug in multi-axis indexing using `.loc` on non-unique indices ([GH6504](https://github.com/pandas-dev/pandas/issues/6504))  
- Bug that caused _ref_locs corruption when slice indexing across columns axis of a DataFrame ([GH6525](https://github.com/pandas-dev/pandas/issues/6525))  
- Regression from 0.13 in the treatment of numpy `datetime64` non-ns dtypes in Series creation ([GH6529](https://github.com/pandas-dev/pandas/issues/6529))  
- `.names` attribute of MultiIndexes passed to `set_index` are now preserved ([GH6459](https://github.com/pandas-dev/pandas/issues/6459)).  
- Bug in setitem with a duplicate index and an alignable rhs ([GH6541](https://github.com/pandas-dev/pandas/issues/6541))  
- Bug in setitem with `.loc` on mixed integer Indexes ([GH6546](https://github.com/pandas-dev/pandas/issues/6546))  
- Bug in `pd.read_stata` which would use the wrong data types and missing values ([GH6327](https://github.com/pandas-dev/pandas/issues/6327))  
- Bug in `DataFrame.to_stata` that lead to data loss in certain cases, and could be exported using the
  wrong data types and missing values ([GH6335](https://github.com/pandas-dev/pandas/issues/6335))  
- `StataWriter` replaces missing values in string columns by empty string ([GH6802](https://github.com/pandas-dev/pandas/issues/6802))  
- Inconsistent types in `Timestamp` addition/subtraction ([GH6543](https://github.com/pandas-dev/pandas/issues/6543))  
- Bug in preserving frequency across Timestamp addition/subtraction ([GH4547](https://github.com/pandas-dev/pandas/issues/4547))  
- Bug in empty list lookup caused `IndexError` exceptions ([GH6536](https://github.com/pandas-dev/pandas/issues/6536), [GH6551](https://github.com/pandas-dev/pandas/issues/6551))  
- `Series.quantile` raising on an `object` dtype ([GH6555](https://github.com/pandas-dev/pandas/issues/6555))  
- Bug in `.xs` with a `nan` in level when dropped ([GH6574](https://github.com/pandas-dev/pandas/issues/6574))  
- Bug in fillna with `method='bfill/ffill'` and `datetime64[ns]` dtype ([GH6587](https://github.com/pandas-dev/pandas/issues/6587))  
- Bug in sql writing with mixed dtypes possibly leading to data loss ([GH6509](https://github.com/pandas-dev/pandas/issues/6509))  
- Bug in `Series.pop` ([GH6600](https://github.com/pandas-dev/pandas/issues/6600))  
- Bug in `iloc` indexing when positional indexer matched `Int64Index` of the corresponding axis and no reordering happened ([GH6612](https://github.com/pandas-dev/pandas/issues/6612))  
- Bug in `fillna` with `limit` and `value` specified  
- Bug in `DataFrame.to_stata` when columns have non-string names ([GH4558](https://github.com/pandas-dev/pandas/issues/4558))  
- Bug in compat with `np.compress`, surfaced in ([GH6658](https://github.com/pandas-dev/pandas/issues/6658))  
- Bug in binary operations with a rhs of a Series not aligning ([GH6681](https://github.com/pandas-dev/pandas/issues/6681))  
- Bug in `DataFrame.to_stata` which incorrectly handles nan values and ignores `with_index` keyword argument ([GH6685](https://github.com/pandas-dev/pandas/issues/6685))  
- Bug in resample with extra bins when using an evenly divisible frequency ([GH4076](https://github.com/pandas-dev/pandas/issues/4076))  
- Bug in consistency of groupby aggregation when passing a custom function ([GH6715](https://github.com/pandas-dev/pandas/issues/6715))  
- Bug in resample when `how=None` resample freq is the same as the axis frequency ([GH5955](https://github.com/pandas-dev/pandas/issues/5955))  
- Bug in downcasting inference with empty arrays ([GH6733](https://github.com/pandas-dev/pandas/issues/6733))  
- Bug in `obj.blocks` on sparse containers dropping all but the last items of same for dtype ([GH6748](https://github.com/pandas-dev/pandas/issues/6748))  
- Bug in unpickling `NaT (NaTType)` ([GH4606](https://github.com/pandas-dev/pandas/issues/4606))  
- Bug in `DataFrame.replace()` where regex meta characters were being treated
  as regex even when `regex=False` ([GH6777](https://github.com/pandas-dev/pandas/issues/6777)).  
- Bug in timedelta ops on 32-bit platforms ([GH6808](https://github.com/pandas-dev/pandas/issues/6808))  
- Bug in setting a tz-aware index directly via `.index` ([GH6785](https://github.com/pandas-dev/pandas/issues/6785))  
- Bug in expressions.py where numexpr would try to evaluate arithmetic ops
  ([GH6762](https://github.com/pandas-dev/pandas/issues/6762)).  
- Bug in Makefile where it didn’t remove Cython generated C files with `make
  clean` ([GH6768](https://github.com/pandas-dev/pandas/issues/6768))  
- Bug with numpy < 1.7.2 when reading long strings from `HDFStore` ([GH6166](https://github.com/pandas-dev/pandas/issues/6166))  
- Bug in `DataFrame._reduce` where non bool-like (0/1) integers were being
  converted into bools. ([GH6806](https://github.com/pandas-dev/pandas/issues/6806))  
- Regression from 0.13 with `fillna` and a Series on datetime-like ([GH6344](https://github.com/pandas-dev/pandas/issues/6344))  
- Bug in adding `np.timedelta64` to `DatetimeIndex` with timezone outputs incorrect results ([GH6818](https://github.com/pandas-dev/pandas/issues/6818))  
- Bug in `DataFrame.replace()` where changing a dtype through replacement
  would only replace the first occurrence of a value ([GH6689](https://github.com/pandas-dev/pandas/issues/6689))  
- Better error message when passing a frequency of ‘MS’ in `Period` construction (GH5332)  
- Bug in `Series.__unicode__` when `max_rows=None` and the Series has more than 1000 rows. ([GH6863](https://github.com/pandas-dev/pandas/issues/6863))  
- Bug in `groupby.get_group` where a datelike wasn’t always accepted ([GH5267](https://github.com/pandas-dev/pandas/issues/5267))  
- Bug in `groupBy.get_group` created by `TimeGrouper` raises `AttributeError` ([GH6914](https://github.com/pandas-dev/pandas/issues/6914))  
- Bug in `DatetimeIndex.tz_localize` and `DatetimeIndex.tz_convert` converting `NaT` incorrectly ([GH5546](https://github.com/pandas-dev/pandas/issues/5546))  
- Bug in arithmetic operations affecting `NaT` ([GH6873](https://github.com/pandas-dev/pandas/issues/6873))  
- Bug in `Series.str.extract` where the resulting `Series` from a single
  group match wasn’t renamed to the group name  
- Bug in `DataFrame.to_csv` where setting `index=False` ignored the
  `header` kwarg ([GH6186](https://github.com/pandas-dev/pandas/issues/6186))  
- Bug in `DataFrame.plot` and `Series.plot`, where the legend behave inconsistently when plotting to the same axes repeatedly ([GH6678](https://github.com/pandas-dev/pandas/issues/6678))  
- Internal tests for patching `__finalize__` / bug in merge not finalizing ([GH6923](https://github.com/pandas-dev/pandas/issues/6923), [GH6927](https://github.com/pandas-dev/pandas/issues/6927))  
- accept `TextFileReader` in `concat`, which was affecting a common user idiom ([GH6583](https://github.com/pandas-dev/pandas/issues/6583))  
- Bug in C parser with leading white space ([GH3374](https://github.com/pandas-dev/pandas/issues/3374))  
- Bug in C parser with `delim_whitespace=True` and `\r`-delimited lines  
- Bug in python parser with explicit MultiIndex in row following column header ([GH6893](https://github.com/pandas-dev/pandas/issues/6893))  
- Bug in `Series.rank` and `DataFrame.rank` that caused small floats (<1e-13) to all receive the same rank ([GH6886](https://github.com/pandas-dev/pandas/issues/6886))  
- Bug in `DataFrame.apply` with functions that used `*args` or `**kwargs` and returned
  an empty result ([GH6952](https://github.com/pandas-dev/pandas/issues/6952))  
- Bug in sum/mean on 32-bit platforms on overflows ([GH6915](https://github.com/pandas-dev/pandas/issues/6915))  
- Moved `Panel.shift` to `NDFrame.slice_shift` and fixed to respect multiple dtypes. ([GH6959](https://github.com/pandas-dev/pandas/issues/6959))  
- Bug in enabling `subplots=True` in `DataFrame.plot` only has single column raises `TypeError`, and `Series.plot` raises `AttributeError` ([GH6951](https://github.com/pandas-dev/pandas/issues/6951))  
- Bug in `DataFrame.plot` draws unnecessary axes when enabling `subplots` and `kind=scatter` ([GH6951](https://github.com/pandas-dev/pandas/issues/6951))  
- Bug in `read_csv` from a filesystem with non-utf-8 encoding ([GH6807](https://github.com/pandas-dev/pandas/issues/6807))  
- Bug in `iloc` when setting / aligning ([GH6766](https://github.com/pandas-dev/pandas/issues/6766))  
- Bug causing UnicodeEncodeError when get_dummies called with unicode values and a prefix ([GH6885](https://github.com/pandas-dev/pandas/issues/6885))  
- Bug in timeseries-with-frequency plot cursor display ([GH5453](https://github.com/pandas-dev/pandas/issues/5453))  
- Bug surfaced in `groupby.plot` when using a `Float64Index` ([GH7025](https://github.com/pandas-dev/pandas/issues/7025))  
- Stopped tests from failing if options data isn’t able to be downloaded from Yahoo ([GH7034](https://github.com/pandas-dev/pandas/issues/7034))  
- Bug in `parallel_coordinates` and `radviz` where reordering of class column
  caused possible color/class mismatch ([GH6956](https://github.com/pandas-dev/pandas/issues/6956))  
- Bug in `radviz` and `andrews_curves` where multiple values of ‘color’
  were being passed to plotting method ([GH6956](https://github.com/pandas-dev/pandas/issues/6956))  
- Bug in `Float64Index.isin()` where containing `nan` s would make indices
  claim that they contained all the things ([GH7066](https://github.com/pandas-dev/pandas/issues/7066)).  
- Bug in `DataFrame.boxplot` where it failed to use the axis passed as the `ax` argument ([GH3578](https://github.com/pandas-dev/pandas/issues/3578))  
- Bug in the `XlsxWriter` and `XlwtWriter` implementations that resulted in datetime columns being formatted without the time ([GH7075](https://github.com/pandas-dev/pandas/issues/7075))
  were being passed to plotting method  
- `read_fwf()` treats `None` in `colspec` like regular python slices. It now reads from the beginning
  or until the end of the line when `colspec` contains a `None` (previously raised a `TypeError`)  
- Bug in cache coherence with chained indexing and slicing; add `_is_view` property to `NDFrame` to correctly predict
  views; mark `is_copy` on `xs` only if its an actual copy (and not a view) ([GH7084](https://github.com/pandas-dev/pandas/issues/7084))  
- Bug in DatetimeIndex creation from string ndarray with `dayfirst=True` ([GH5917](https://github.com/pandas-dev/pandas/issues/5917))  
- Bug in `MultiIndex.from_arrays` created from `DatetimeIndex` doesn’t preserve `freq` and `tz` ([GH7090](https://github.com/pandas-dev/pandas/issues/7090))  
- Bug in `unstack` raises `ValueError` when `MultiIndex` contains `PeriodIndex` ([GH4342](https://github.com/pandas-dev/pandas/issues/4342))  
- Bug in `boxplot` and `hist` draws unnecessary axes ([GH6769](https://github.com/pandas-dev/pandas/issues/6769))  
- Regression in `groupby.nth()` for out-of-bounds indexers ([GH6621](https://github.com/pandas-dev/pandas/issues/6621))  
- Bug in `quantile` with datetime values ([GH6965](https://github.com/pandas-dev/pandas/issues/6965))  
- Bug in `Dataframe.set_index`, `reindex` and `pivot` don’t preserve `DatetimeIndex` and `PeriodIndex` attributes ([GH3950](https://github.com/pandas-dev/pandas/issues/3950), [GH5878](https://github.com/pandas-dev/pandas/issues/5878), [GH6631](https://github.com/pandas-dev/pandas/issues/6631))  
- Bug in `MultiIndex.get_level_values` doesn’t preserve `DatetimeIndex` and `PeriodIndex` attributes ([GH7092](https://github.com/pandas-dev/pandas/issues/7092))  
- Bug in `Groupby` doesn’t preserve `tz` ([GH3950](https://github.com/pandas-dev/pandas/issues/3950))  
- Bug in `PeriodIndex` partial string slicing ([GH6716](https://github.com/pandas-dev/pandas/issues/6716))  
- Bug in the HTML repr of a truncated Series or DataFrame not showing the class name with the large_repr set to ‘info’
  ([GH7105](https://github.com/pandas-dev/pandas/issues/7105))  
- Bug in `DatetimeIndex` specifying `freq` raises `ValueError` when passed value is too short ([GH7098](https://github.com/pandas-dev/pandas/issues/7098))  
- Fixed a bug with the info repr not honoring the display.max_info_columns setting ([GH6939](https://github.com/pandas-dev/pandas/issues/6939))  
- Bug `PeriodIndex` string slicing with out of bounds values ([GH5407](https://github.com/pandas-dev/pandas/issues/5407))  
- Fixed a memory error in the hashtable implementation/factorizer on resizing of large tables ([GH7157](https://github.com/pandas-dev/pandas/issues/7157))  
- Bug in `isnull` when applied to 0-dimensional object arrays ([GH7176](https://github.com/pandas-dev/pandas/issues/7176))  
- Bug in `query`/`eval` where global constants were not looked up correctly
  ([GH7178](https://github.com/pandas-dev/pandas/issues/7178))  
- Bug in recognizing out-of-bounds positional list indexers with `iloc` and a multi-axis tuple indexer ([GH7189](https://github.com/pandas-dev/pandas/issues/7189))  
- Bug in setitem with a single value, MultiIndex and integer indices ([GH7190](https://github.com/pandas-dev/pandas/issues/7190), [GH7218](https://github.com/pandas-dev/pandas/issues/7218))  
- Bug in expressions evaluation with reversed ops, showing in series-dataframe ops ([GH7198](https://github.com/pandas-dev/pandas/issues/7198), [GH7192](https://github.com/pandas-dev/pandas/issues/7192))  
- Bug in multi-axis indexing with > 2 ndim and a MultiIndex ([GH7199](https://github.com/pandas-dev/pandas/issues/7199))  
- Fix a bug where invalid eval/query operations would blow the stack ([GH5198](https://github.com/pandas-dev/pandas/issues/5198))  



<a id='whatsnew-0-14-0-contributors'></a>

## Contributors

A total of 94 people contributed patches to this release.  People with a
“+” by their names contributed a patch for the first time.


- Acanthostega +  
- Adam Marcus +  
- Alex Gaudio  
- Alex Rothberg  
- AllenDowney +  
- Andrew Rosenfeld +  
- Andy Hayden  
- Antoine Mazières +  
- Benedikt Sauer  
- Brad Buran  
- Christopher Whelan  
- Clark Fitzgerald  
- DSM  
- Dale Jung  
- Dan Allan  
- Dan Birken  
- Daniel Waeber  
- David Jung +  
- David Stephens +  
- Douglas McNeil  
- Garrett Drapala  
- Gouthaman Balaraman +  
- Guillaume Poulin +  
- Jacob Howard +  
- Jacob Schaer  
- Jason Sexauer +  
- Jeff Reback  
- Jeff Tratner  
- Jeffrey Starr +  
- John David Reaver +  
- John McNamara  
- John W. O’Brien  
- Jonathan Chambers  
- Joris Van den Bossche  
- Julia Evans  
- Júlio +  
- K.-Michael Aye  
- Katie Atkinson +  
- Kelsey Jordahl  
- Kevin Sheppard +  
- Matt Wittmann +  
- Matthias Kuhn +  
- Max Grender-Jones +  
- Michael E. Gruen +  
- Mike Kelly  
- Nipun Batra +  
- Noah Spies +  
- PKEuS  
- Patrick O’Keeffe  
- Phillip Cloud  
- Pietro Battiston +  
- Randy Carnevale +  
- Robert Gibboni +  
- Skipper Seabold  
- SplashDance +  
- Stephan Hoyer +  
- Tim Cera +  
- Tobias Brandt  
- Todd Jennings +  
- Tom Augspurger  
- TomAugspurger  
- Yaroslav Halchenko  
- agijsberts +  
- akittredge  
- ankostis +  
- anomrake  
- anton-d +  
- bashtage +  
- benjamin +  
- bwignall  
- cgohlke +  
- chebee7i +  
- clham +  
- danielballan  
- hshimizu77 +  
- hugo +  
- immerrr  
- ischwabacher +  
- jaimefrio +  
- jreback  
- jsexauer +  
- kdiether +  
- michaelws +  
- mikebailey +  
- ojdo +  
- onesandzeroes +  
- phaebz +  
- ribonoous +  
- rockg  
- sinhrks +  
- unutbu  
- westurner  
- y-p  
- zach powers  